In [1]:
pip install pandas psycopg2 sqlalchemy pyarrow


In [41]:
from sqlalchemy import create_engine, text, ARRAY, Integer, Text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd

credentials = "C:\\Users\\Kory\\OneDrive\\Documents\\Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db, conn

def insert_feather_into_pg(feather_filepath, table_name, db_schema="public"):
    # Read the Feather file into a DataFrame
    df = pd.read_feather(feather_filepath)

    # Convert columns with array data to PostgreSQL array format
    df['sub_opinions'] = df['sub_opinions'].apply(lambda x: list(x) if isinstance(x, pd.Series) else x)
    df['opinions_text'] = df['opinions_text'].apply(lambda x: list(x) if isinstance(x, pd.Series) else x)

    # Connect to the PostgreSQL database
    db, conn = pgconnect(credentials)

    if db is not None:
        try:
            # Create the table schema in PostgreSQL
            create_table_statement = text(f"""
                CREATE TABLE IF NOT EXISTS {db_schema}.{table_name} (
                    cluster_id INTEGER,
                    sub_opinions INTEGER[],
                    date_filed DATE,
                    case_name TEXT,
                    scdb_votes_majority FLOAT,
                    scdb_votes_minority FLOAT,
                    opinion_id INTEGER,
                    author_id INTEGER,
                    opinions_cited INTEGER,
                    opinions_text TEXT[],
                    date_dob DATE,
                    political_party CHAR(1),
                    date_start DATE,
                    date_left DATE,
                    docket_id INTEGER,
                    decision TEXT,
                    full_name TEXT
                );
            """)
            conn.execute(create_table_statement)
            # Insert the data into the PostgreSQL database
            df.to_sql(table_name, db, schema=db_schema, if_exists='replace', index=False, dtype={'sub_opinions': ARRAY(Integer), 'opinions_text': ARRAY(Text)})  # Use 'append' to add to existing data
            print(f"Data inserted successfully into {table_name} table.")
        except Exception as e:
            print("Error while inserting data into the database:")
            print(e)
        finally:
            conn.close()
            print("Connection closed.")
    else:
        print("Database connection failed.")

In [47]:
import pandas as pd
df = pd.read_feather("C:\\Users\\Kory\\OneDrive\\Documents\\cleaned_data.feather")
df.to_excel("file.xlsx")

In [42]:
import numpy as np
from psycopg2.extensions import register_adapter, AsIs

def addapt_numpy_array(numpy_array):
    return AsIs(tuple(numpy_array))

def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)

def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)


register_adapter(np.int64, addapt_numpy_int64)
register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.ndarray, addapt_numpy_array)

In [43]:
insert_feather_into_pg("C:\\Users\\Kory\\OneDrive\\Documents\\cleaned_data.feather", "feather2")

Connected successfully.
Data inserted successfully into feather2 table.
Connection closed.
